# Notebook 4: Modelos generativos

## Pre-requisitos

### Instalar paquetes

Si la práctica requiere algún paquete de Python, habrá que incluir una celda en la que se instalen. Si usamos un paquete que se ha utilizado en prácticas anteriores, podríamos dar por supuesto que está instalado pero no cuesta nada satisfacer todas las dependencias en la propia práctica para reducir las dependencias entre ellas.

In [ ]:
# Ejemplo de instalación de tensorflow 2.0
#%tensorflow_version 2.x
# !pip3 install tensorflow  # NECESARIO SOLO SI SE EJECUTA EN LOCAL
import tensorflow as tf

# Hacemos los imports que sean necesarios
import numpy as np
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import accuracy_score

# Modelos generativos sobre MNIST

Lo primero que tenemos que hacer es cargar el dataset.

In [ ]:
labeled_data = 0.01 # Vamos a usar el etiquetado de sólo el 1% de los datos
np.random.seed(42)

(x_train, y_train), (x_test, y_test), = tf.keras.datasets.mnist.load_data()

indexes = np.arange(len(x_train))
np.random.shuffle(indexes)
ntrain_data = int(labeled_data*len(x_train))
unlabeled_train = x_train[indexes[ntrain_data:]]
x_train = x_train[indexes[:ntrain_data]]
y_train = y_train[indexes[:ntrain_data]]

In [ ]:
# TODO: Haz el preprocesado que necesites aquí (si lo necesitas)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1]*x_train.shape[2]))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1]*x_test.shape[2]))
unlabeled_train = np.reshape(unlabeled_train, (unlabeled_train.shape[0], unlabeled_train.shape[1]*unlabeled_train.shape[2]))

one_hot_train = np.zeros((y_train.size, len(set(y_train))), dtype=int)
one_hot_train[np.arange(y_train.size), y_train ] = 1

one_hot_test = np.zeros((y_test.size, len(set(y_test))), dtype=int)
one_hot_test[np.arange(y_test.size), y_test ] = 1

## Modelo generativo

Vamos a crear nuestro propio modelo generativo. En clase de teoría has visto muchas versiones distintas:

1. Mezcla de distribuciones de Gaussianas (GMM)
1. Mezcla de distribuciones multinomiales (Naive Bayes)
1. Modelos de Markov ocultos (HMM)

Tal y como se os apunta en teoría, los modelos generativos abordan un problema más general, y aprenden realmente cómo se estructuran y distribuyen los datos de entrada. 

En nuestro caso, vamos a distribuír los datos de entrada mediante el uso de **Autoencoders**. 

# Autoencoders

El autoencoder es un tipo de red que se utiliza para aprender codificaciones eficientes de datos sin etiquetar (lo que se conoce como aprendizaje no supervisado). Es una red que tiene el mismo tamaño en la entrada como en la salida, puesto que el objetivo de la red es reconstruír la entrada con la menor pérdida posible.

Si lo que hacemos es reconstruír la entrada, ¿qué sentido tiene el usar la red? Habitualmente, **la red consta, a su mitad, de una capa con menos elementos que los datos de entrada**. Por tanto, al reconstruír los datos de la entrada a la salida, en esa capa tendremos una versión *comprimida* de la entrada, que contendrá la mayor parte de su información.

Por tanto, podemos dividir un autoencoder en 3 secciones diferentes, tal y como se ve en la siguiente figura:

![autoencoder](https://drive.google.com/uc?export=view&id=1yxkKZV0J0YplQAGPGJxQ2Z80Ad6L94eu)


1. **Encoder:** es la parte inicial de la red, encargada de comprimir los datos de la entrada.
1. **Code:** es la salida del encoder, contiene la versión *comprimida* de los datos de entrada.
1. **Decoder:** se encarga de, partiendo de la salida del *Encoder*, reconstruír la red.

## Crea tu propio Autoencoder

El diseño del autoencoder es libre (capas densas, convolucionales, ...), puedes crearlo como quieras. **El único requisito es que tiene que mantener los nombres (y parámetros) de las funciones descritas abajo.**

In [ ]:
# TODO: crea tu propio autoencoder


class MiAutoencoder:

    def __init__(self, input_shape):
        
        self.input_shape = input_shape
        
        self.encoder = models.Sequential()
        self.encoder.add(layers.InputLayer(shape=self.input_shape))
        self.encoder.add(layers.Dense(512, activation='relu'))
        self.encoder.add(layers.Dense(128, activation='relu'))
        self.encoder.add(layers.Dense(64, activation='relu'))
        self.encoder.add(layers.Dense(32, activation='relu'))
        self.encoder.add(layers.Dense(32, activation='relu'))  # Latent space

        self.decoder = models.Sequential()
        self.decoder.add(layers.InputLayer(shape=(32,)))
        self.decoder.add(layers.Dense(32, activation='relu'))
        self.decoder.add(layers.Dense(64, activation='relu'))
        self.decoder.add(layers.Dense(128, activation='relu'))
        self.decoder.add(layers.Dense(512, activation='relu'))
        self.decoder.add(layers.Dense(self.input_shape[0], activation='sigmoid'))  # Output should match input

        self.autoencoder = models.Sequential([self.encoder, self.decoder])

        self.autoencoder.compile(optimizer='adam', loss='mse')
    
    def fit(self, X, y=None, sample_weight=None, batch_size=60_000, epochs=20):
        # TODO: entrena el modelo. Escoge el tamaño de batch y el número de epochs que quieras
        
        self.autoencoder.fit(X, X, 
                             batch_size=batch_size, 
                             epochs=epochs, 
                             sample_weight=sample_weight)

    def get_encoded_data(self, X):
        # TODO: devuelve la salida del encoder (code)
        
        return self.encoder.predict(X)
        
    def __del__(self):
        # elimina todos los modelos que hayas creado
        tf.keras.backend.clear_session() # Necesario para liberar la memoria en GPU

## Crea tu propio Clasificador

El diseño del clasificador es libre, pero recuerda que tiene que ser simple (máximo dos capas). **El único requisito es que tiene que mantener los nombres (y parámetros) de las funciones descritas abajo.**

In [ ]:
# TODO: crea tu propio clasificador

class MiClasificador:

    def __init__(self):
        # TODO : define el modelo y compílalo
        
        self.input_shape = (32,)
        
        self.classifier = models.Sequential()
        self.classifier.add(layers.InputLayer(shape=self.input_shape))
        self.classifier.add(layers.Dense(32, activation='relu'))
        self.classifier.add(layers.Dense(10, activation='sigmoid'))

        self.classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    def fit(self, X, y, sample_weight=None, batch_size=60_000, epochs=15):
        # TODO: entrena el modelo. Escoge el tamaño de batch y el número de epochs que quieras
        self.classifier.fit(X, y, 
                             batch_size=batch_size, 
                             epochs=epochs, 
                             sample_weight=sample_weight)

    def predict(self, X):
        # TODO: devuelve la clase ganadora

        return np.argmax(self.predict_proba(X)) + 1
    
    def predict_proba(self, X):
        
        return self.classifier.predict(X)
    
    def score(self, X, y):
        
        return self.classifier.evaluate(X, y)[1]

    def __del__(self):
        # elimina todos los modelos que hayas creado
        tf.keras.backend.clear_session() # Necesario para liberar la memoria en GPU

### Entrenamiendo del modelo semisupervisado

El entrenamiento del sistema semisupervisado se realiza en dos pasos.

1. Se entrena el autoencoder con todos los datos (etiquetados y sin etiquetar).
1. Se entrena un clasificador simple (una o dos capas), teniendo como entrada la salida del encoder (**code**) de los datos etiquetados.

<font color='red'>NOTA:</font> para entrenar (y predecir) vamos a utilizar los nombres de las funciones que hemos definido en el autoencoder y en el clasificador.

In [ ]:
# TODO: implementa el algoritmo semisupervised_training.

def semisupervised_training(autoencoder, classifier, x_train, y_train, unlabeled_data):

    all_x = np.vstack((x_train, unlabeled_train))
    autoencoder.fit(all_x)
    x_coded = autoencoder.get_encoded_data(x_train)
    classifier.fit(x_coded, y_train)

### Entrenamos nuestro modelo

Usa lo hecho anteriormente para entrenar tu clasificador de una manera semi-supervisada.

In [ ]:
# Crea tu autoencoder y tu clasificador

autoencoder = MiAutoencoder(input_shape=x_train[0].shape)
classifier = MiClasificador()

In [ ]:
# TODO: Entrena tu modelo

semisupervised_training(autoencoder=autoencoder, classifier=classifier, x_train=x_train, y_train=one_hot_train, unlabeled_data=unlabeled_train)

In [ ]:
# TODO: Obtén la precisión sobre el conjunto de test
pred_data = autoencoder.get_encoded_data(x_test)
print('Test accuracy :', classifier.score(pred_data, one_hot_test))

## Mejorando el código

nuestro modelo actual requiere de dos pasos para entrenarse, pero podría realizarse en un único paso si **creamos un modelo con las dos salidas (autoencoder y clasificador)**. 

Para ello, hay que tener en cuenta que, en los datos sin etiquetar, su contribución al clasificador debería ser nula.


### TRABAJO: Crea el nuevo modelo y modifica la función semisupervised_training para tener en cuenta todos los puntos mencionados anteriormente

In [ ]:
# TODO: crea el nuevo modelo

# TODO: crea tu propio clasificador

class MiClasificadorSemisupervisado:

    def __init__(self, input_shape):
        # TODO : define el modelo y compílalo
        
        self.input_shape = input_shape
        self.num_classes = 10
        
        input_layer = layers.Input(shape=self.input_shape)
        
        # Encoder part (shared for both autoencoder and classifier)
        encoded = layers.Dense(128, activation='relu', kernel_regularizer='l2')(input_layer)
        encoded = layers.Dense(64, activation='relu', kernel_regularizer='l2')(encoded)
        encoded = layers.Dense(32, activation='relu', kernel_regularizer='l2')(encoded)
        
        # Decoder for autoencoder part
        decoded = layers.Dense(64, activation='relu', kernel_regularizer='l2')(encoded)
        decoded = layers.Dense(128, activation='relu', kernel_regularizer='l2')(decoded)
        decoded = layers.Dense(self.input_shape[0], activation='sigmoid',name='autoencoder')(decoded)

        # Classifier part
        classifier = layers.Dense(32, activation='relu', kernel_regularizer='l2')(encoded)
        classifier = layers.Dense(32, activation='relu', kernel_regularizer='l2')(classifier)
        classifier_output = layers.Dense(self.num_classes, activation='softmax',name='classifier')(classifier)

        # Autoencoder model (for reconstructing input)
        self.autoencoder = models.Model(input_layer, decoded)
        self.autoencoder.compile(optimizer='adam', loss='mse')
        
        # Classifier model (for predicting class labels)
        self.classifier = models.Model(input_layer, classifier_output)
        self.classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Combined model with two outputs: one for autoencoder (reconstruction) and one for classifier (classification)
        self.model = models.Model(input_layer, 
                                  [decoded, classifier_output])
                                  #classifier_output)
        self.model.compile(optimizer='adam',
                           loss=['mse', 'categorical_crossentropy'],
                           #loss='categorical_crossentropy',
                           loss_weights=[.5, 1.5],  # Adjust loss weights if needed
                           metrics=['accuracy', 'accuracy'])
    
    def fit(self, X, y, unlabeled_data, batch_size,  epochs):
        # TODO: entrena el modelo. Escoge el tamaño de batch y el número de epochs que quieras, y define bien el sample_weight

        all_x = np.vstack((X, unlabeled_train))
        y_zeros = np.zeros((unlabeled_data.shape[0],y.shape[1]))
        all_y = np.vstack((y,y_zeros))
        weight_autoencoder = np.ones(len(all_x))
        weight_classifier = np.array([1]*len(X) + [0]*len(unlabeled_data))
        
        h = self.model.fit(all_x, 
                       [all_x, all_y], 
                       #all_y,
                       sample_weight=[weight_autoencoder, weight_classifier], 
                       #sample_weight=sample_weight,
                       epochs=epochs, 
                       batch_size=batch_size, 
                       verbose=1)
        return h

    def predict(self, X):
        # TODO: devuelve la clase ganadora del clasificador
        _, predictions = self.model.predict(X)
        return predictions.argmax(axis=1)
    
    def predict_proba(self, X):
        # TODO: devuelve la probabilidad del clasificador
        _, predictions = self.model.predict(X)
        return predictions
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

    def __del__(self):
        # elimina todos los modelos que hayas creado
        tf.keras.backend.clear_session() # Necesario para liberar la memoria en GPU

In [ ]:
# TODO: reescribe la función semisupervised_training para incorporar las mejoras mencionadas anteriormente

model = MiClasificadorSemisupervisado(input_shape=x_train[0].shape)

def semisupervised_training_v2(model, x_train, y_train, unlabeled_data):
    h = model.fit(x_train, y_train, unlabeled_data, batch_size=20_000, epochs = 100)
    return h

In [ ]:
# TODO: Crea y entrena tu clasificador

model = MiClasificadorSemisupervisado(input_shape=x_train[0].shape)

h = semisupervised_training_v2(model, x_train, one_hot_train, unlabeled_train)

In [ ]:
# TODO: Obtén la precisión sobre el conjunto de test
print('Test accuracy :', model.score(x_test, y_test))

# Hay vida más allá del autoencoder

¿Has probado a utilizar otro método distinto del autoencoder para obtener una respresentación similar a la salida del encoder? La idea es la siguiente:

1. Define un modelo $model$ convolucional similar al encoder de un autoencoder (la entrada es el tamaño de la imagen, la salida el vector de representación)
1. Define una capa de salida $cluster$ que, partiendo de la salida de model, nos devuelva una salida con el mismo número de clases que el dataset a utilizar (la entrada es el vector de representación), usando softmax como activación de salida
1. Para cada batch de entrenamiento $X$:  # Usa un batch alto, mínimo 128
  1. Modifica las imágenes de entrada con [data_augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation?hl=es-419), llámala $augX_1$.
  1. Modifica otra vez las imágenes de entrada con [data_augmentation_2](https://www.tensorflow.org/tutorials/images/data_augmentation?hl=es-419), llámala $augX_2$.
  1. $augX_{1comp} \leftarrow model(augX_1)$
  1. $augX_{2comp} \leftarrow model(augX_2)$
  1. $cX_{1comp} \leftarrow cluster(augX_{1comp})$
  1. $cX_{2comp} \leftarrow cluster(augX_{2comp})$
  1. $M \leftarrow augX_{1comp} ~ augX_{2comp}^T$
  1. $loss_C \leftarrow cX_{1comp}(1 - cX_{1comp}) + cX_{2comp}(1 - cX_{2comp})$ # Puede que tengas que crear tu [propia función de coste](https://keras.io/api/losses/#creating-custom-losses)
  1. $loss_M \leftarrow crossentropy(I, softmax(M/\tau, axis=1)))$ # Puede que tengas que crear tu [propia función de coste](https://keras.io/api/losses/#creating-custom-losses)
  1. $\tau$ es un hiperparámetro que se suele definir a 5.0
  1. $loss \leftarrow loss_M + \lambda~loss_C$
    1. $\lambda$ es un hiperparámetro (puedes probar con 0.5)


In [ ]:
# Escribe aquí la solución. Crea tantos bloques de código como necesites. Puedes utilizar la siguiente red para generar distorsiones

data_augmentation = tf.keras.models.Sequential(
    [
        # tf.keras.layers.RandomFlip("horizontal"),  # Puede ser util en otros casos
        tf.keras.layers.RandomRotation(0.05),
        tf.keras.layers.RandomTranslation(0.15, 0.15),
        tf.keras.layers.RandomZoom(.15),
    ]
)

data_augmentation_2 = tf.keras.models.Sequential(
    [
        # tf.keras.layers.RandomFlip("horizontal"),  # Puede ser util en otros casos
        tf.keras.layers.RandomTranslation(0.15, 0.15),
        tf.keras.layers.Resizing(48, 48), # para CIFAR, para MNIST usar 40 en lugar de 48
        tf.keras.layers.RandomCrop(32, 32), # para CIFAR, para MNIST usar 28 en lugar de 32
    ]
)


# Trabajo extra

¿Has probado a hacer el autoencoder totalmente convolucional? Para el *decoder* puedes usar las funciones [UpSampling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling2D) o [Conv2DTranspose](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose).

In [ ]:
import matplotlib.pyplot as plt
plt.plot(h.history['classifier_accuracy']);